In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch


from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.data_sets.shape_net import pc_loader as sn_pc_loader
from tf_lab.nips.helper import load_shape_net_models_used_by_wu

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
do_training = True
first_time_running = False
load_model = True
z_rotate = True
seed = 42
max_training_epochs = 1610
loss = 'emd'
is_convolutional = True

In [5]:
if is_convolutional:
    arch_tag = 'convolutional_arch'
    
    encoder_args = {'n_filters': [128, 128, 256, 512],
                    'filter_sizes': [40, 20, 10, 10],
                    'strides': [1, 2, 2, 1]
                   }
        
else:
    arch_tag = 'mlp_arch'
    encoder_args = {'n_filters': [64, 128, 256, 512],
                    'filter_sizes' :[1, 1, 1, 1],
                    'strides': [1, 1, 1, 1]
                   }

In [6]:
experiment_name = 'wu_classes_rotated_' + arch_tag + '_' + str(n_pc_samples) +  'pts_' + loss
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/wu_classes_rotated_convolutional_arch_2048pts_emd'

In [7]:
full_pclouds, full_model_names, full_syn_ids = load_shape_net_models_used_by_wu(n_pc_samples, full_pclouds_path)
model_unames = full_model_names + '.' + full_syn_ids
train_data = PointCloudDataSet(full_pclouds, labels=model_unames)

airplane 02691156
4045 files containing complete point clouds were found.
car 02958343
7497 files containing complete point clouds were found.
chair 03001627
6778 files containing complete point clouds were found.
sofa 04256520
3173 files containing complete point clouds were found.
rifle 04090263
2372 files containing complete point clouds were found.
boat 02858304
1137 files containing complete point clouds were found.
table 04379243
8509 files containing complete point clouds were found.


In [8]:
train_data.num_examples

33511

In [22]:
# syn_id = '04530566' # vessel -> use this next time instead of the "boat"

1939


In [9]:
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf    
else:
    n_input = [n_pc_samples, 3]
    
    decoder_args = {'layer_sizes': [1024, 2048, np.prod(n_input)] }
    
    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = z_rotate,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args,        
               )

    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'filter_sizes': [40, 20, 10, 10], 'n_filters': [128, 128, 256, 512], 'strides': [1, 2, 2, 1]}
               experiment_name: wu_classes_rotated_convolutional_arch_2048pts_emd
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: emd
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/mo

In [10]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)

Model restored in epoch 2320.


In [ ]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '2321', 'training time (minutes)=', '2.3797', 'loss=', '69.329087107')
('Epoch:', '2322', 'training time (minutes)=', '2.4262', 'loss=', '69.090369102')
('Epoch:', '2323', 'training time (minutes)=', '2.4328', 'loss=', '68.646129728')
('Epoch:', '2324', 'training time (minutes)=', '2.4403', 'loss=', '69.293102077')
('Epoch:', '2325', 'training time (minutes)=', '2.4436', 'loss=', '69.605623729')
('Epoch:', '2326', 'training time (minutes)=', '2.4582', 'loss=', '68.854093791')
('Epoch:', '2327', 'training time (minutes)=', '2.4535', 'loss=', '69.349353460')
('Epoch:', '2328', 'training time (minutes)=', '2.4512', 'loss=', '69.416071661')
('Epoch:', '2329', 'training time (minutes)=', '2.4328', 'loss=', '69.125604521')
('Epoch:', '2330', 'training time (minutes)=', '2.4539', 'loss=', '69.126784026')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/wu_classes_rotated_convolutional_arch_2048pts_emd/models.ckpt-2330 is not in all_model_check